# read

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('./data/train_df.csv', encoding='cp949')
test  = pd.read_csv('./data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('./data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


In [4]:
test.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,32000,5013000043028400,1165000021097200,식품,농산물
1,32001,5013000044016100,1154500002066400,식품,농산물
2,32002,5013000205030200,4139000102013200,식품,농산물
3,32003,5013000205030200,4221000040093400,식품,농산물
4,32004,5013000268011400,2726000004017100,식품,농산물


# preprocessing

In [5]:
train['ag'] = 0
test['ag']  = 0

train.loc[train['DL_GD_MCLS_NM'] == '농산물', 'ag'] = 1
test.loc[test['DL_GD_MCLS_NM'] == '농산물', 'ag'] = 1

In [6]:
large_label = dict(train.groupby('DL_GD_LCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
mid_label   = dict(train.groupby('DL_GD_MCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

train['DL_GD_LCLS_NM'] = train['DL_GD_LCLS_NM'].map(large_label)
test['DL_GD_LCLS_NM']  = test['DL_GD_LCLS_NM'].map(large_label)

train['DL_GD_MCLS_NM'] = train['DL_GD_MCLS_NM'].map(mid_label)
test['DL_GD_MCLS_NM']  = test['DL_GD_MCLS_NM'].map(mid_label)

In [7]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,0
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,0
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,0
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,1
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,0


In [8]:
# 격자공간고유번호 슬라이싱
slice_range = range(3, 11)

for i in slice_range:
    train['send_' + str(i)] = train['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['send_' + str(i)]  = test['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')

for i in slice_range:
    train['rec_' + str(i)] = train['REC_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['rec_' + str(i)]  = test['REC_SPG_INNB'].astype('str').str[:i].astype('int64')

In [9]:
train['eq'] = 0
test['eq']  = 0

train.loc[train['SEND_SPG_INNB'] == train['REC_SPG_INNB'], 'eq'] = 1
test.loc[test['SEND_SPG_INNB'] == test['REC_SPG_INNB'], 'eq'] = 1

In [10]:
threshold = 20

temp_send = train.groupby('SEND_SPG_INNB').min()
temp_rec = train.groupby('REC_SPG_INNB').min()

send_idx = temp_send[temp_send['INVC_CONT'] >= threshold].index
rec_idx  = temp_rec[temp_rec['INVC_CONT'] >= threshold].index

train[(train['SEND_SPG_INNB'].isin(send_idx)) | (train['REC_SPG_INNB'].isin(rec_idx))]

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,eq
158,158,5011000319046400,2635000026058300,4.658195,4.297401,33,1,501,5011,50110,501100,5011000,50110003,501100031,5011000319,263,2635,26350,263500,2635000,26350000,263500002,2635000026,0
162,162,5011000373058300,4615000790058400,4.658195,5.780338,36,0,501,5011,50110,501100,5011000,50110003,501100037,5011000373,461,4615,46150,461500,4615000,46150007,461500079,4615000790,0
202,202,5011000767075300,2729000007038300,4.658195,8.196172,25,0,501,5011,50110,501100,5011000,50110007,501100076,5011000767,272,2729,27290,272900,2729000,27290000,272900000,2729000007,0
605,605,1111000025032200,5011000373070400,6.678694,10.375479,37,0,111,1111,11110,111100,1111000,11110000,111100002,1111000025,501,5011,50110,501100,5011000,50110003,501100037,5011000373,0
647,647,4159000310093100,5011000494074200,4.658195,6.130548,22,0,415,4159,41590,415900,4159000,41590003,415900031,4159000310,501,5011,50110,501100,5011000,50110004,501100049,5011000494,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31504,31504,1117000009037100,5011000315059300,6.547718,6.547718,24,0,111,1117,11170,111700,1117000,11170000,111700000,1117000009,501,5011,50110,501100,5011000,50110003,501100031,5011000315,0
31675,31675,5011001053094200,4613000323074400,4.658195,4.297401,20,1,501,5011,50110,501100,5011001,50110010,501100105,5011001053,461,4613,46130,461300,4613000,46130003,461300032,4613000323,0
31915,31915,5011000137023400,2638000022073100,4.658195,6.807151,50,0,501,5011,50110,501100,5011000,50110001,501100013,5011000137,263,2638,26380,263800,2638000,26380000,263800002,2638000022,0
31950,31950,5011000319076400,5011000319076400,4.658195,5.647691,25,0,501,5011,50110,501100,5011000,50110003,501100031,5011000319,501,5011,50110,501100,5011000,50110003,501100031,5011000319,1


In [11]:
# train['outlier'] = 0
# test['outlier'] = 0
# train.loc[(train['SEND_SPG_INNB'].isin(send_idx)) | (train['REC_SPG_INNB'].isin(rec_idx)), 'outlier'] = 1
# test.loc[(test['SEND_SPG_INNB'].isin(send_idx)) | (test['REC_SPG_INNB'].isin(rec_idx)), 'outlier'] = 1

In [12]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,eq
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,0,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220,0
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,0,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178,0
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,0,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265,0
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,1,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315,0
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,0,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177,0


# encoding

In [13]:
train.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)
test.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)

In [14]:
train.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,eq
0,6.678694,3.672897,3,0,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220,0
1,5.407921,4.444134,3,0,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178,0
2,6.678694,10.375479,9,0,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265,0
3,4.658195,4.297401,10,1,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315,0
4,4.658195,6.807151,3,0,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177,0


# modeling

In [15]:
X = train.drop('INVC_CONT', axis=1)
y = train['INVC_CONT']

In [16]:
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

model = CatBoostRegressor(random_state=0, random_strength=0)
model.fit(X, y)
pd.DataFrame({'col':X.columns, 'fi':model.feature_importances_})

Learning rate set to 0.070793
0:	learn: 5.7073632	total: 199ms	remaining: 3m 19s
1:	learn: 5.6666338	total: 347ms	remaining: 2m 53s
2:	learn: 5.6532495	total: 492ms	remaining: 2m 43s
3:	learn: 5.6184380	total: 663ms	remaining: 2m 45s
4:	learn: 5.5866445	total: 804ms	remaining: 2m 39s
5:	learn: 5.5774475	total: 937ms	remaining: 2m 35s
6:	learn: 5.5688931	total: 1.02s	remaining: 2m 25s
7:	learn: 5.5407524	total: 1.1s	remaining: 2m 16s
8:	learn: 5.5145200	total: 1.24s	remaining: 2m 16s
9:	learn: 5.5080119	total: 1.32s	remaining: 2m 11s
10:	learn: 5.4848033	total: 1.53s	remaining: 2m 17s
11:	learn: 5.4794073	total: 1.61s	remaining: 2m 12s
12:	learn: 5.4578611	total: 1.67s	remaining: 2m 7s
13:	learn: 5.4381481	total: 1.79s	remaining: 2m 6s
14:	learn: 5.4339285	total: 1.91s	remaining: 2m 5s
15:	learn: 5.4298201	total: 2.08s	remaining: 2m 8s
16:	learn: 5.4259115	total: 2.14s	remaining: 2m 3s
17:	learn: 5.4226875	total: 2.19s	remaining: 1m 59s
18:	learn: 5.4034731	total: 2.23s	remaining: 1m 55

,col,fi
0,DL_GD_LCLS_NM,3.291525
1,DL_GD_MCLS_NM,22.770690
2,ag,0.107509
3,send_3,1.546645
4,send_4,8.756201
5,send_5,11.351777
6,send_6,0.000000
7,send_7,9.509897
8,send_8,12.437513
9,send_9,4.504164


In [17]:
drop_cols = ['ag', 'send_3', 'send_6', 'rec_3', 'rec_5', 'rec_6', 'rec_7', 'rec_10', 'eq']
X.drop(drop_cols, axis=1, inplace=True)
X.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,send_4,send_5,send_7,send_8,send_9,send_10,rec_4,rec_8,rec_9
0,6.678694,3.672897,1129,11290,1129000,11290000,112900001,1129000014,5011,50110002,501100022
1,5.407921,4.444134,1135,11350,1135000,11350000,113500000,1135000009,5011,50110001,501100017
2,6.678694,10.375479,1135,11350,1135000,11350000,113500003,1135000030,5011,50110002,501100026
3,4.658195,4.297401,1154,11545,1154500,11545000,115450000,1154500002,5011,50110003,501100031
4,4.658195,6.807151,1165,11650,1165000,11650000,116500002,1165000021,5011,50110001,501100017


In [18]:
model = CatBoostRegressor(random_state=0, random_strength=0)
model.fit(X, y)
pd.DataFrame({'col':X.columns, 'fi':model.feature_importances_})

Learning rate set to 0.070793
0:	learn: 5.7073632	total: 68.3ms	remaining: 1m 8s
1:	learn: 5.6936588	total: 129ms	remaining: 1m 4s
2:	learn: 5.6554759	total: 150ms	remaining: 49.7s
3:	learn: 5.6193061	total: 172ms	remaining: 42.8s
4:	learn: 5.5873681	total: 192ms	remaining: 38.3s
5:	learn: 5.5607715	total: 243ms	remaining: 40.3s
6:	learn: 5.5520626	total: 257ms	remaining: 36.5s
7:	learn: 5.5447180	total: 271ms	remaining: 33.6s
8:	learn: 5.5382790	total: 284ms	remaining: 31.3s
9:	learn: 5.5120812	total: 296ms	remaining: 29.3s
10:	learn: 5.5066491	total: 311ms	remaining: 27.9s
11:	learn: 5.4828801	total: 340ms	remaining: 28s
12:	learn: 5.4778013	total: 380ms	remaining: 28.8s
13:	learn: 5.4563040	total: 397ms	remaining: 28s
14:	learn: 5.4363226	total: 416ms	remaining: 27.3s
15:	learn: 5.4323093	total: 445ms	remaining: 27.4s
16:	learn: 5.4143502	total: 458ms	remaining: 26.5s
17:	learn: 5.4109037	total: 474ms	remaining: 25.9s
18:	learn: 5.3929042	total: 489ms	remaining: 25.3s
19:	learn: 5.3

,col,fi
0,DL_GD_LCLS_NM,3.311332
1,DL_GD_MCLS_NM,22.193128
2,send_4,10.727913
3,send_5,9.541331
4,send_7,6.795890
5,send_8,12.543380
6,send_9,13.989902
7,send_10,3.800089
8,rec_4,4.476950
9,rec_8,7.770929


In [19]:
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor(random_state=0, random_strength=0)

param = {
    
}

gscv = GridSearchCV(
    estimator=model, param_grid=param, scoring='neg_root_mean_squared_error', 
    refit=True, cv=5
)

gscv.fit(X, y)

print(gscv.best_score_)

Learning rate set to 0.068341
0:	learn: 5.4330044	total: 79ms	remaining: 1m 18s
1:	learn: 5.4204753	total: 103ms	remaining: 51.6s
2:	learn: 5.4094725	total: 192ms	remaining: 1m 3s
3:	learn: 5.3993725	total: 270ms	remaining: 1m 7s
4:	learn: 5.3910256	total: 364ms	remaining: 1m 12s
5:	learn: 5.3833809	total: 404ms	remaining: 1m 7s
6:	learn: 5.3768203	total: 450ms	remaining: 1m 3s
7:	learn: 5.3710255	total: 521ms	remaining: 1m 4s
8:	learn: 5.3489921	total: 613ms	remaining: 1m 7s
9:	learn: 5.3437721	total: 640ms	remaining: 1m 3s
10:	learn: 5.3235792	total: 747ms	remaining: 1m 7s
11:	learn: 5.3192640	total: 800ms	remaining: 1m 5s
12:	learn: 5.3150992	total: 841ms	remaining: 1m 3s
13:	learn: 5.3110121	total: 909ms	remaining: 1m 4s
14:	learn: 5.2930033	total: 946ms	remaining: 1m 2s
15:	learn: 5.2766635	total: 1.04s	remaining: 1m 4s
16:	learn: 5.2621889	total: 1.14s	remaining: 1m 6s
17:	learn: 5.2594569	total: 1.24s	remaining: 1m 7s
18:	learn: 5.2464642	total: 1.34s	remaining: 1m 9s
19:	learn:

In [22]:
pred = gscv.predict(test)
sub['INVC_CONT'] = pred

In [23]:
sub.head()

,index,INVC_CONT
0,32000,4.343780
1,32001,4.679269
2,32002,5.261931
3,32003,6.178804
4,32004,4.806881


In [24]:
# sub.to_csv('./sub/logistics_08.csv', index=False)